# 语言模型

In [1]:
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
dataset = Dataset.load_from_disk("datas/wiki_cn_filtered")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("models/bloom-389")

In [4]:
dataset

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
def process_func(examples):
    examples = [sentence + tokenizer.eos_token for sentence in examples["completion"]]
    return tokenizer(examples, max_length=384, truncation=True)

In [6]:
tokenized_data = dataset.map(process_func, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
tokenized_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [8]:
print(tokenized_data[0])

{'input_ids': [13110, 34800, 13535, 916, 33156, 10, 256, 576, 387, 479, 681, 5453, 10955, 915, 24124, 5317, 13110, 6573, 20757, 13535, 355, 5358, 1490, 583, 28056, 1407, 3855, 671, 6113, 189, 6732, 4302, 9488, 3434, 6900, 1322, 355, 37336, 9825, 4608, 13461, 1359, 5358, 355, 5317, 13110, 34800, 4433, 7189, 25722, 29747, 13110, 1498, 12047, 6347, 23563, 2139, 2066, 420, 29288, 25, 15, 7635, 39288, 355, 1484, 5835, 6272, 23, 15, 4180, 39288, 355, 5358, 4516, 11621, 23, 15, 10641, 4887, 1712, 420, 2450, 31163, 8085, 11621, 5358, 553, 9888, 2731, 21335, 5358, 553, 9876, 14011, 4434, 5358, 553, 21484, 4514, 17170, 25871, 8085, 5358, 553, 17489, 6945, 11097, 13535, 641, 33623, 1484, 5835, 1689, 2063, 5358, 569, 5835, 671, 16225, 3422, 189, 13, 23158, 27894, 33227, 1022, 11396, 3347, 1813, 1504, 6566, 1813, 355, 9155, 8633, 1504, 2063, 1813, 189, 13, 23158, 813, 7817, 5358, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [33]:
model = AutoModelForCausalLM.from_pretrained("models/bloom-389")

In [34]:
args = TrainingArguments(
    output_dir="trained/model_for_causalLM",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=30,
    num_train_epochs=1,
    fp16=True,
    optim="adafactor",
    learning_rate=1e-5,
    weight_decay=0.01
)

In [35]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [51]:
trainer.train()

Step,Training Loss
30,3.220300
60,3.230300
90,3.289800
120,3.273900
150,3.287400
180,3.328700
210,3.369700
240,3.394500
270,3.375800
300,3.393900


TrainOutput(global_step=313, training_loss=3.322041033175045, metrics={'train_runtime': 253.629, 'train_samples_per_second': 39.428, 'train_steps_per_second': 1.234, 'total_flos': 6102526349377536.0, 'train_loss': 3.322041033175045, 'epoch': 1.0})

In [37]:
from transformers import BloomForCausalLM, DataCollatorForLanguageModeling

In [38]:
from transformers import pipeline

In [53]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [66]:
pipe("西安博物馆", max_new_tokens=128, do_sample=True)

[{'generated_text': '西安博物馆（ึชญญ머จฮษญำ）是中国西安的一座博物馆。该博物馆由中国西安市文物局投资、西安市文物博物馆管理，西安市博物馆协会管理。该博物馆位于西安市碑林区南大街10号。\n历史\n该博物馆始建于1969年，是西安市最早建立的文物博物馆，也是中国西安市博物馆协会的所在地。1996年，该博物馆被列为全国第一批重点文物保护单位。2004年，该博物馆被列为首批全国重点文物保护单位。2009年，该博物馆被列为首批全国重点文物保护单位。2010年，该博物馆被列为第一批全国重点文物保护单位。2013年'}]

# 掩码语言模型

In [1]:
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [4]:
dataset = Dataset.load_from_disk("datas/wiki_cn_filtered")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("models/macbert-base")

In [6]:
def process_func(examples):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

In [7]:
tokenized_data = dataset.map(process_func, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForMaskedLM.from_pretrained("models/macbert-base")

Some weights of the model checkpoint at models/macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
args = TrainingArguments(
    output_dir="trained/model_for_maskedlLM",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=30,
    num_train_epochs=1,
    fp16=True,
    optim="adafactor",
    learning_rate=1e-5,
    weight_decay=0.01
)

In [10]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True)
)

In [11]:
trainer.train()

Step,Training Loss
30,1.437700
60,1.372300
90,1.343900
120,1.354700
150,1.317700
180,1.307000
210,1.291900
240,1.328700
270,1.329900
300,1.315400


TrainOutput(global_step=313, training_loss=1.3359024303789717, metrics={'train_runtime': 173.8249, 'train_samples_per_second': 57.529, 'train_steps_per_second': 1.801, 'total_flos': 1872398224800768.0, 'train_loss': 1.3359024303789717, 'epoch': 1.0})

In [12]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [14]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9988579750061035,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi ' an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0008164722239598632,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi ' an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 9.377801325172186e-05,
   'token': 2339,
   'token_str': '工',
   'sequence': "[CLS] 西 安 交 通 工 [MASK] 博 物 馆 （ xi ' an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 6.345337169477716e-05,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi ' an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 1.5071497728058603e-05,
   'token': 2825,
   'token_str': '技',
   'sequence': "[CLS] 西 安 交 通 技 [MASK] 博 物 馆 （ xi ' an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.99